In [13]:
import re
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('averaged_perceptron_tagger')

from string import punctuation

punctuation += '«»—…'

!pip install pymystem3 pymorphy2

from pymystem3 import Mystem

m = Mystem()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
with open('our_corpus.txt', encoding="utf-8") as data:
    text = str(data.read())
# не было возможности норм отедлить конспекты друг от друга,
# так что вручную проставила перед каждым новым конспектом %
# и потом разделила по этому символу (больше его в тексте не было)
docs = text.split('%')

# тут был какой то странный наблр символов в начале мб кодировка (удалила)
docs = docs[1:]

In [15]:
# тут мы проходимя по каждому доку (док=1 конспект) во всех 34 доках и далее:
# делим док на предложения через sent_tokinize
# делим док на заголовок (doc_name = text[1])
# выделяем гендер (первый символ в доке, проставлен вручную (gender = doc[0]))
# собственно текст конспекта = sentences_pre = text[2:],
# те все предложения кроме первых двух (кроме заголовка и пола)

genders = []
doc_ids = []
doc_names = []
doc_texts = []
sent_ids = []
sentence_texts = []
sent_positions = []
word_ids = []
words = []
lemmas = []
tags = []
word_positions = []

sent_id = 0
word_id = 0
sentences_all = []

for doc in docs:

  text = sent_tokenize(doc)
  doc_name = text[1]
  doc_text = doc
  gender = doc[0]
  sentences_pre = text[2:]
  sentences = []

  # это доп. деление тк часто в конпектах люди ипользуют не точки,
  # а списки чтобы членить мысль
  for sen in sentences_pre:
    sen = sen.split('\n')
    for i in sen:
      i = i.split('\n*')
      for d in i:
        sentences.append(d)

  # итого в списке sentrnces лежат предложения для нашего разбора

  for s in sentences:
    sentence_text = s

    word_position = 0

    sent_position = sentences.index(s) + 1
    sent_id += 1

    sentences_all.append(s)

    # тут разбор майстема и добавление во все списки всех нужных для датафрейма вещей
    for w in m.analyze(s):
      if 'analysis' in w:
        if w['analysis'] != []: # майстем не анализирует пунктуацию и слова латиницей (то есть здесь мы отбираем только кориллические токены)
          word_id += 1
          word_position += 1

          word = w['text']
          lemma = w['analysis'][0]['lex']
          gr = w['analysis'][0]['gr']
          pos = gr.split('=')[0].split(',')[0]

          tags.append(pos)
          words.append(word.lower())
          lemmas.append(lemma.lower())
          doc_names.append(doc_name)
          doc_texts.append(doc_text)
          sentence_texts.append(sentence_text)
          sent_positions.append(sent_position)
          sent_ids.append(sent_id)
          word_positions.append(word_position)
          genders.append(gender)
          doc_ids.append(docs.index(doc) + 1)
          word_ids.append(word_id)

        # если токен не кириллический то мы отбираем то что не пунктуация (.isalpha()) и добавляем тег LAT
        else:
          if w['text'].isalpha():
            word_id += 1
            word_position += 1

            word = w['text']
            lemma = m.lemmatize(w['text'])[0]
            pos = 'LAT'

            tags.append(pos)
            words.append(word.lower())
            lemmas.append(lemma.lower())
            doc_names.append(doc_name)
            doc_texts.append(doc_text)
            sentence_texts.append(sentence_text)
            sent_positions.append(sent_position)
            word_positions.append(word_position)
            genders.append(gender)
            doc_ids.append(docs.index(doc) + 1)
            sent_ids.append(sent_id)
            word_ids.append(word_id)

In [ ]:
print(len(words))

10022


In [ ]:
print(len(sentences_all))

1144


In [16]:
# функция для получения обратных индексов словоформ
inverted_word_indexes = []

for w in words:
  inverted_index = []
  for s in sentences_all:
    if w.lower() in s.lower():
      inverted_index.append(sentences_all.index(s) + 1)
  inverted_word_indexes.append(inverted_index)

In [17]:
# лемматизированные предложения нужны для обратных индексов лемм
lemmatized_sentences_all = []

for s in sentences_all:
  ana_s = m.analyze(s)
  words_for_s = []
  for w in ana_s:
    if 'analysis' in w:
      if w['analysis'] != []:
        words_for_s.append(w['analysis'][0]['lex'])
      else:
        words_for_s.append(m.lemmatize(w['text'])[0])

  lemmatized_sentences_all.append(' '.join(words_for_s))

In [ ]:
len(lemmatized_sentences_all)

1144

In [18]:
# функция для получения обратных индексов лемм
inverted_lemma_indexes = []

for w in lemmas:
  inverted_index = []
  for s in lemmatized_sentences_all:
    if w.lower() in s.lower():
      inverted_index.append(lemmatized_sentences_all.index(s) + 1)
  inverted_lemma_indexes.append(inverted_index)

In [19]:
# собираем датафрейм и переводим его из словаря питона в датафрейм пандас
corpus_dict = {
    'gender': genders,
    'doc_id': doc_ids,
    'doc_name': doc_names,
    'doc_text': doc_texts,
    'sent_id': sent_ids,
    'sentence_text': sentence_texts,
    'sent_position': sent_positions,
    'word_id': word_ids,
    'word': words,
    'lemma': lemmas,
    'tag': tags,
    'word_position': word_positions,
    'inverted_word_index': inverted_word_indexes,
    'inverted_lemma_index': inverted_lemma_indexes
}

corpus_df = pd.DataFrame.from_dict(corpus_dict)

In [20]:
corpus_df

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
0,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,1,из-за,из-за,PR,1,"[1, 54, 171, 991]","[1, 54, 171, 991]"
1,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,2,того,то,SPRO,2,"[1, 7, 12, 135, 147, 153, 235, 365, 399, 412, ...","[1, 2, 6, 7, 11, 12, 19, 21, 29, 43, 44, 52, 5..."
2,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,3,что,что,CONJ,3,"[1, 19, 58, 71, 135, 141, 147, 235, 277, 292, ...","[1, 19, 58, 71, 135, 141, 147, 235, 277, 292, ..."
3,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,4,смыслы,смысл,S,4,"[1, 265, 267, 746]","[1, 17, 38, 42, 48, 104, 265, 267, 394, 395, 4..."
4,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,5,не,не,PART,5,"[1, 2, 7, 8, 14, 17, 18, 19, 32, 36, 43, 47, 5...","[1, 2, 7, 8, 14, 17, 18, 19, 32, 36, 43, 47, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10018,значения,значение,S,16,"[7, 21, 79, 98, 135, 222, 426, 432, 443, 476, ...","[7, 14, 21, 79, 98, 105, 108, 131, 135, 142, 2..."
10018,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10019,или,или,CONJ,17,"[17, 18, 58, 105, 119, 120, 129, 142, 167, 170...","[17, 18, 58, 105, 119, 120, 129, 167, 170, 230..."
10019,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10020,формы,форма,S,18,[1144],"[9, 38, 41, 163, 167, 177, 213, 301, 303, 488,..."
10020,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10021,составных,составной,A,19,[1144],[1144]


In [ ]:
corpus_df[corpus_df.word_id == 1425]

,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,word_position
1424,8,Семантические роли.,f 8. Семантические роли. Автор понятия. Основн...,205,"Бывают и более длинные списки ролей, см.",62,1425,см,7


In [ ]:
corpus_df.to_csv('nlp_project_corpus_df.csv')

In [22]:
LATs = corpus_df[corpus_df.tag == 'LAT']
LATs

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
23,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,3,Перелом случился в XX веке.,3,24,xx,xx,LAT,4,"[3, 4, 27]","[3, 4, 27]"
26,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,4,до XX века:,4,27,xx,xx,LAT,2,"[3, 4, 27]","[3, 4, 27]"
43,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,6,- толковые одноязычные словари (XIX век): этим...,6,44,xix,xix,LAT,4,[6],[6]
129,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,17,"- Нет смысла, есть реакция на стимул (языковой...",17,130,ср,ср,LAT,10,"[17, 54, 73, 98, 125, 130, 195, 199, 201, 277,...","[17, 54, 73, 98, 125, 130, 195, 199, 201, 277,..."
156,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,19,"- Чтобы понять, что такое соленое, надо быть х...",19,157,nacl,nacl,LAT,16,[19],[19]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9667,f,33,Импликатура как механизм семантических сдвигов.,f 33. Импликатура как механизм семантических с...,1103,В некоторых употреблениях идея настоящего или ...,34,9668,etc,etc,LAT,23,[1103],[1103]
9789,f,33,Импликатура как механизм семантических сдвигов.,f 33. Импликатура как механизм семантических с...,1112,"В контексте грамматики можно ещё говорить, нап...",43,9790,нсв,нсв,LAT,12,[1112],[1112]
9927,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1135,"3) фразеологические сочетания (выражения, в ко...",21,9928,ср,ср,LAT,18,"[17, 54, 73, 98, 125, 130, 195, 199, 201, 277,...","[17, 54, 73, 98, 125, 130, 195, 199, 201, 277,..."
9944,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1137,Разная степень композициональности и устойчиво...,23,9945,vs,vs,LAT,9,"[25, 117, 199, 202, 309, 393, 846, 950, 952, 9...","[25, 117, 199, 202, 309, 393, 846, 950, 952, 9..."


In [ ]:
nltk.pos_tag(['≈'])

[('≈', 'NN')]

In [23]:
lat_words = LATs.word.unique()

In [24]:
lat_words

array(['xx', 'xix', 'ср', 'nacl', 'vs', 'дж', 'specific', 'definite',
       'indefinite', 'non', 'ground', 'figure', 'trajector', 'landmark',
       'langacker', 'go', 'gone', 'steal', 'rob', 'f', 'or', 'the', 'act',
       'of', 'cooking', 'a', 'cook', 'food', 'being', 'cooked',
       'container', 'to', 'hold', 'while', 'it', 'is', 'heating',
       'instrument', 'framenet', 'apply', 'heat', 'frame', 'elements',
       'fe', 'lexical', 'units', 'lu', 'placing', 'theme', 'goal',
       'agent', 'cause', 'revenge', 'offender', 'injury', 'injured',
       'party', 'avenger', 'punishment', 'born', 'locative', 'relation',
       'examination', 'examples', 'sentences', 'relations', 'core',
       'child', 'time', 'place', 'means', 'commerce', 'goods', 'transfer',
       'seller', 'buyer', 'purpose', 'commercial', 'event', 'valences',
       'valence', 'patterns', 'inheritance', 'mapping', 'parent',
       'perspectivization', 'buy', 'sell', 'criminal', 'process',
       'arrest', 'trial',

In [ ]:
corpus_df.tag.unique()

array(['PR', 'SPRO', 'S', 'PART', 'V', 'A', 'ADV', 'ADVPRO', 'LAT',
       'CONJ', 'APRO', 'INTJ', 'NUM', 'ANUM'], dtype=object)

In [ ]:
latposes = []

for i in lat_words:
  if i.isalpha():
    latposes.append(nltk.pos_tag([i])[0])

In [ ]:
latposes

[('xx', 'NN'),
 ('xix', 'NN'),
 ('ср', 'NN'),
 ('nacl', 'NN'),
 ('дж', 'NN'),
 ('specific', 'JJ'),
 ('definite', 'NN'),
 ('indefinite', 'NN'),
 ('ground', 'NN'),
 ('vs', 'NN'),
 ('figure', 'NN'),
 ('trajector', 'NN'),
 ('landmark', 'NN'),
 ('langacker', 'NN'),
 ('go', 'VB'),
 ('gone', 'VBN'),
 ('steal', 'NN'),
 ('rob', 'NN'),
 ('f', 'NN'),
 ('or', 'CC'),
 ('the', 'DT'),
 ('act', 'NN'),
 ('of', 'IN'),
 ('cooking', 'NN'),
 ('a', 'DT'),
 ('cook', 'NN'),
 ('food', 'NN'),
 ('being', 'VBG'),
 ('cooked', 'NNS'),
 ('container', 'NN'),
 ('to', 'TO'),
 ('hold', 'NN'),
 ('while', 'IN'),
 ('it', 'PRP'),
 ('is', 'VBZ'),
 ('heating', 'NN'),
 ('instrument', 'NN'),
 ('framenet', 'NN'),
 ('frame', 'NN'),
 ('elements', 'NNS'),
 ('fe', 'NN'),
 ('lexical', 'JJ'),
 ('units', 'NNS'),
 ('lu', 'NN'),
 ('placing', 'VBG'),
 ('theme', 'NN'),
 ('goal', 'NN'),
 ('agent', 'NN'),
 ('cause', 'NN'),
 ('revenge', 'NN'),
 ('offender', 'NN'),
 ('injury', 'NN'),
 ('injured', 'JJ'),
 ('party', 'NN'),
 ('avenger', 'NN'),
 (

In [ ]:
set(latposes)

{'CC',
 'DT',
 'IN',
 'JJ',
 'NN',
 'NNS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBZ'}